In [1]:
from lxml import html
from bs4 import BeautifulSoup
import requests
import urllib.request
import bs4
import csv
import pandas as pd

In [3]:
# create list of the links in movies
tree = html.parse(r"movies2.html")
movies2 = html.tostring(tree)
soup = BeautifulSoup(movies2)
links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

In [5]:
for i in range(len(links)+1):
    try:
        html = urllib.request.urlopen(links[i]).read() 
        with open('../data_input/article_{}.html'.format(i), 'wb+') as f:
                    f.write(html)
    except Exception as e:
        print(i, links[i], e)
        continue

KeyboardInterrupt: 

In [12]:
# save all .tsv
# made some test
for article in range(10000):  # range of files
    try:
        html = BeautifulSoup(open("../data_input/article_{}.html".format(article)), 'html.parser')
    except Exception as e:   # if article doesn't exists
        print(article, e)
        continue

    title = html.select("h1")[0].text
        
    # initialize tmp as intro
    tmp = 'intro'
    sections = {'intro' : '', 'plot' : ''}
    
    # take all paragraphs section by section and save only intro and plot
    for section in html.select('div.mw-parser-output > *'):
        if(section.name == 'p' and tmp == 'intro'):
            sections['intro'] += section.text.strip()
    
        # chage tmp on section names
        if(section.name in ['h2','h3']):
            tmp = section.span['id']
    
        # take only sections we are interrested in
        if(section.name == 'p' and tmp in ['Plot','Plot_summary','Premiese']):
            sections['plot'] += section.text.strip()
    
    # we doesn't take in consideration pages without Plot
    if(sections['plot'] == ''):
        print(article, 'No Plot')
        continue

    # dictionary for infobox
    d = {'film_name':title, 'Directed by': 'NA', 'Produced by': 'NA', 'Written by': 'NA', 'Starring':'NA', 'Music by': 'NA',
         'Release date': 'NA', 'Running time': 'NA', 'Country': 'NA', 'Language': 'NA', 'Budget': 'NA'}

    # take elem from infobox
    info_box = html.findAll(['th', 'td'])
    for elem in info_box:
        info = elem.text.strip('\n')   # take text from the table
        if info in d:
            d[info] = info_box[info_box.index(elem)+1].text.strip('\n')
    
    # select elem in oroder as a list to save in .tsv
    ld = list(d.values())
    columns = ['title', 'intro', 'plot', 'film_name', 'Directed by', 'Produced by', 'Written by', 'Starring', 'Music by',
         'Release date', 'Running time', 'Country', 'Language', 'Budget']
    data = [title,sections['intro'], sections['plot']] + ld[1:] 

    # create and save a tsv
    with open('../data_output/article_{}.tsv'.format(article), 'w', newline='',encoding='utf-8') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerow(columns)
        tsv_output.writerow(data)

9960 [Errno 2] No such file or directory: '../data_input/article_9960.html'
9961 [Errno 2] No such file or directory: '../data_input/article_9961.html'
9962 [Errno 2] No such file or directory: '../data_input/article_9962.html'
9963 [Errno 2] No such file or directory: '../data_input/article_9963.html'


In [9]:
b = pd.read_csv('../data_output/article_0.tsv', sep = '\t')
print(b['plot'][0])

Warren Stacy (Gene Davis) is a young office equipment repairman who kills women after they reject his sexual advances. His attempts at flirting are always seen as "creepy" by women, resulting in frequent rejections.[4] His first victim is Betty (June Gilbert), an office worker of his acquaintance. He tracks her down to a wooded area, and observes her having sex with her boyfriend. He ambushes the couple, kills the boyfriend and then gives chase to the naked woman. He catches her and stabs her to death.[4]Two Los Angeles police detectives, Leo Kessler (Charles Bronson) and Paul McAnn (Andrew Stevens), investigate his murders. Kessler is a seasoned veteran of the force, while McAnn is considerably younger.[4] Stacy avoids prosecution by constructing sound alibis and assaulting his victims while naked except for a pair of latex gloves to hide fingerprints, thus minimizing evidence.Laurie Kessler (Lisa Eilbacher) is the only daughter of Leo and an acquaintance to some of the victims. A stu